# Dateningestierung

Die Daten werden nun aus den Parquet Files geladen und dann in die Datenbank überführt

In [23]:
import pandas as pd
from dotenv import load_dotenv
import os
from src.models import db, Station, StationName, StationLocation, Wind10Minutes, model_init
import numpy as np

load_dotenv()

# Datenmodel initalisieren. Dabei werden alle vorherigen Daten gelöscht.
# model_init()

True

In [24]:
PARQUET_FOLDER = os.path.join(os.getenv('PROJECT_DIR'), 'data','interim', 'parquet' )

GEO_PARQUET_FILE_PATH = os.path.join(PARQUET_FOLDER, 'geo.parquet')
NAME_PARQUET_FILE_PATH = os.path.join(PARQUET_FOLDER, 'name.parquet')
WIND_PARQUET_FILE_PATH = os.path.join(PARQUET_FOLDER, 'wind.parquet')



In [25]:
geo_df = pd.read_parquet(GEO_PARQUET_FILE_PATH)
name_df = pd.read_parquet(NAME_PARQUET_FILE_PATH)


In [26]:
wind_df = pd.read_parquet(WIND_PARQUET_FILE_PATH, engine='pyarrow')
wind_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2895037 entries, 0 to 157823
Data columns (total 5 columns):
 #   Column         Dtype         
---  ------         -----         
 0   station_id     int64         
 1   timestamp      datetime64[ns]
 2   quality        int64         
 3   avg_speed      float64       
 4   avg_direction  float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 132.5 MB


In [27]:
with db.atomic():
    
    print(f"Deleted {Wind10Minutes.delete().execute()} wind data")
    print(f"Deleted {StationName.delete().execute()} station names")
    print(f"Deleted {StationLocation.delete().execute()} station locations")    
    print(f"Deleted {Station.delete().execute()} stations")
    


Deleted 0 wind data
Deleted 450 station names
Deleted 0 station locations
Deleted 310 stations


In [28]:

# insert stations
stations_data = [{'id': sid} for sid in geo_df['station_id'].unique().tolist()]
Station.insert_many(stations_data).execute()
print(f"Inserted {Station.select().count()} stations")

# insert station names
station_names_data = name_df.replace({np.nan: None}).to_dict(orient="records")
StationName.insert_many(station_names_data).execute()
print(f"Inserted {StationName.select().count()} station names")

# insert station locations
station_locations_data = geo_df.drop(columns=['station_name', 'state']).replace({np.nan: None}).to_dict(orient="records")
StationLocation.insert_many(station_locations_data).execute()
print(f"Inserted {StationLocation.select().count()} station locations")


Inserted 310 stations
Inserted 450 station names
Inserted 1264 station locations


In [29]:
#insert wind data
wind_data = wind_df.replace({np.nan: None}).rename(columns={'station_id': 'station'}).to_dict(orient="records")
print(f"Will insert {len(wind_data)} data points")

# split into chunks of 1000
wind_data_splitted = [wind_data[i:i + 10000] for i in range(0, len(wind_data), 10000)]
print(f"Deleted {Wind10Minutes.delete().execute()} wind data")
for i, wind in enumerate(wind_data_splitted):
    Wind10Minutes.insert_many(wind).execute()
    print(f"inserted {(i+1)*10000} of {len(wind_data)} data points", end="\r")
#Wind10Minutes.insert_many(wind_data).execute()
print(f"Inserted {Wind10Minutes.select().count()} wind data")

Will insert 2895037 data points
Deleted 0 wind data
Inserted 2895037 wind data7 data points


In [30]:
wind_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2895037 entries, 0 to 157823
Data columns (total 5 columns):
 #   Column         Dtype         
---  ------         -----         
 0   station_id     int64         
 1   timestamp      datetime64[ns]
 2   quality        int64         
 3   avg_speed      float64       
 4   avg_direction  float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 132.5 MB
